# Základy měření s termokamerou

Na tomto cvičení budeme používat kamery [Tau2](https://www.flir.com/products/tau-2/) od firmy FLIR. 

Tau2 je schopno několika módů snímání. V prvním módu snímá veličinu, která je lineární s radiometrickým fluxem.
V druhém módu (který budeme používat my) snímá veličinu lineární s teplotou scény. Výhoda druhého modelu je taková, že kompenzuje vliv teploty jádra na naměřenou teplotu scény.

Kamery snímají infračervené záření o vlnové délce ~7.5 - ~13.5 μm, jinými slovy měří [radiometrický flux](https://en.wikipedia.org/wiki/Radiant_flux). Vztah mezi fluxem a zdánlivou povrchovou teplotou je popsaný ve funkcích `flux_to_temp` a `temp_to_flux`. Pro jednoduchost teplotu měříme v Kelvinech.

Parametry `R B F O` jsou fitnuté při kalibraci kamery, pro nás nemají žádný význam (pro zájemce [Planck's law](https://en.wikipedia.org/wiki/Planck%27s_law)).

Popíšeme model, který kamera používá, aby z celkového naměřeného fluxu (scene flux) vyrobila povrchovou teplotu měřeného objektu.
Kamera snímá celkový flux scény, ten se skládá z:
- flux vyzářený měřeným objektem
- flux vyzářený dalšími objekty ve scéně

Měřený flux je dále ovlivněn atmosférou, která část pohltí a část sama vyzáří.
V poslední řadě se aplikuje vliv filtru, který část záření pohltí a část odrazí.
Celkový flux, který naměří kamera nazýváme scene flux.

Kamera provádí postprocessing, aby ze scene flux vyrobila flux měřeného objektu.
Model si vyrobíme na tomto cvičení.

Platí:
$$
1 = \tau + r + \varepsilon,
$$
kde $\tau$ je transmisivita (propustnost), $r$ je reflexivita (odrazivost) a $\varepsilon$ je emisivita. 

Známé hodnoty emisivity (7.5 - 13.5 μm):
https://www.engineeringtoolbox.com/emissivity-coefficients-d_447.html

Známé hodnoty transmisivity (7.5 - 13.5 μm):
- germániové sklo: 0.4
- germániové sklo s antireflexním coatingem: 0.95

![Radiometrický model](./images/radiometric_model.png)

## Měření propustnosti fólie

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import improutils as iu

### Pomocné funkce 

In [ ]:
def flux_to_temp(val):
    return (
        sensor_params.B / np.log(sensor_params.R / 
        (val - sensor_params.O) + sensor_params.F)
    )


def temp_to_flux(val):
    return (
        sensor_params.R / 
        (np.exp(sensor_params.B / val) - sensor_params.F) + sensor_params.O
    )


def tlin_to_temp(val):
    return val * 0.04


def atmospheric_transmissivity(atm_temp_C, humidity, distance_m):
    K1 = 1.5587e+0
    K2 = 6.9390e-2
    K3 = -2.7816e-4
    K4 = 6.8455e-7

    # alpha = 0.01262
    alpha = 0
    beta = -0.00667

    # NOTE: Tayloruv polynom 3. radu
    exponent = (
        K1 + K2 * atm_temp_C + 
        K3 * atm_temp_C**2 + K4 * atm_temp_C**3
    )

    sqrt_H2O = np.sqrt(humidity * np.exp(exponent))
    sqrt_distance_m = np.sqrt(distance_m)
    return np.exp(sqrt_distance_m * (alpha + beta * sqrt_H2O))

### Úkol


#### 1) Nasnímejte nahřáté baterie termokamerou.
 Zapojte nejdřív první sadu a vyčkejte 1 minutu, než zapojíte druhou sadu baterií. Pořiďte 2 snímky - jednu bez folie a jednu s folií.

**Zapište** si hodnoty **R, B, F, O** z TauPlayeru

#### 2) Načtěte si snímky a zobrazte ve stupních Celsia
Obrázky z termokamery nemají standardní rozsah [0...255]. Pro správné načtení se vhodně využije flag ```cv2.IMREAD_ANYDEPTH```.

Hodnoty z termokamery škálou neodpovídají stupním Celsia (jsou mnohem větší). Pro správné naškálování využijte funkci ```tlin_to_temp()```. Výsledné hodnoty jsou však v Kelvinech a je třeba je převést na stupně Celsia.

In [ ]:
no_filter_image = cv2.imread(..., ...)
with_filter_image = cv2.imread(..., ...)

def imshow(*args, **kwargs):
    ## Plot image with colorbar
    plt.imshow(*args, **kwargs)
    plt.colorbar()


kwargs = {'vmin': 0,  'cmap': 'viridis'}
plt.figure(figsize=(13, 8))
plt.subplot(1, 2, 1)
imshow(..., **kwargs)
plt.subplot(1, 2, 2)
imshow(..., **kwargs)

### 3) Vykreslete graf závislosti transmisivity atmosféry na vzdálenosti a na vlhkosti

Použijte funkci atmospheric_transmisivity.

In [ ]:
for humidity in ...(0.1, 0.9, num=9): # preferably linear spacing
    distances = ...(-1, 3, num=10) # preferably logaritmic spacing
    transmissivities = ...(20, humidity, distances)
    plt.plot(distances, 100 * transmissivities, label=f'{humidity:.1f}')
    plt.xlabel('distance (m)')
    plt.ylabel('atm_trans (%)')
    plt.legend()
plt.show()

### 4) Seznamte se s modelelem pro odhad propustnosti filtru

#### Model

In [ ]:
class SensorParameters:
    R = ...
    B = ...
    F = ...
    O = ...

sensor_params = SensorParameters()

class RadiometricModel:
    obj_trans = 0  # zanedbáváme
    atm_reflex = 0  # zanedbáváme
    win_emis = 0  # zanedbáváme

    def __init__(self, *, obj_emis=0.95, bkg_temp=293.15,
            atm_temp=293.15, atm_trans=0.95, 
            win_temp=293.15, win_trans=0.95):
        # object
        self.obj_emis = obj_emis
        self.obj_reflex = 1 - obj_emis

        # background
        self.bkg_temp = bkg_temp

        # atmosphere
        self.atm_temp = atm_temp  
        self.atm_trans = atm_trans
        self.atm_emis = 1 - atm_trans

        # window
        self.win_temp = win_temp
        self.win_trans = win_trans
        self.win_reflex = 1 - win_trans

### 5) Naimplementujte funkce, které budou sloužit na výpočet `obj_flux` a `scene_flux`.

$F_1$ je flux vyzářený ze součástky, $F_2$ je flux vyzářený skrz atmosféru a $F_3$ je výsledný flux scény.

$$ F_1 = F_{obj} \varepsilon _{obj} + F_{bkg}r_{obj}  $$
$$ F_2 = F_{1} \tau _{atm} + F_{atm}\varepsilon_{atm}  $$
$$ F_3 = F_{2} \tau _{win} + F_{win}r_{win}  $$

In [ ]:
def obj_flux_to_scene_flux(model: RadiometricModel, obj_flux):
    win_flux = ...
    atm_flux = ...
    bkg_flux = ...
    return ...

def scene_flux_to_obj_flux(model: RadiometricModel, scene_flux):
    win_flux = ...
    atm_flux = ...
    bkg_flux = ...
    return ...

### 6) Změřte propustnost filtru na objektivu a vytvořte model, který bude kompenzovat vliv filtru na výsledných snímcích.

In [ ]:
camera_model = RadiometricModel()

img_nofilter_obj_flux = ...(...(no_filter_image))
img_filter_obj_flux = ...(...(with_filter_image))
img_nofilter_scene_flux = ...(camera_model, img_nofilter_obj_flux)
img_filter_scene_flux = ...(camera_model, img_filter_obj_flux)

print(f'No filter raw: MEAN = {img_nofilter_scene_flux.mean()}, MAX = {img_nofilter_scene_flux.max()}')
print(f'Filter raw: MEAN = {img_filter_scene_flux.mean()}, MAX = {img_filter_scene_flux.max()}')

print(f'Diff raw of maximas: {img_nofilter_scene_flux.max() - img_filter_scene_flux.max()}',
      f'({(100 * (img_nofilter_scene_flux.max() - img_filter_scene_flux.max()) / img_nofilter_scene_flux.max())})%')

imshow((img_nofilter_scene_flux - img_filter_scene_flux) / img_nofilter_scene_flux)
plt.title('Transmissivity diff (%)')

Na základě vypočtené hodnoty změny fluxu vytvořte model, který bude kompenzovat vliv filtru.

A porovnejte teplotní maxima původního a vykompenzovaného obrázku.


In [ ]:
updated_model = RadiometricModel(win_trans=...)

img_corrected = ...

imshow(img_corrected)

print(f'Maximum of original image without filter: {tlin_to_temp(no_filter_image).max()-273.15} °C')
print(f'Maximum of the compensated image with filter: {...} °C')
